In [0]:
from google.colab import drive
import os
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [3]:
print(f"CUDA is available? {torch.cuda.is_available()}")
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(dev)

CUDA is available? True
cuda


In [4]:
drive.mount("/content/drive",True)
root_dir = "/content/drive/My Drive/SB3/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_transform = T.Compose([
    T.Resize(256),
    T.ToTensor()
])

In [0]:
train_dataset = ImageFolder(os.path.join(root_dir, "train"), transform=train_transform)
test_dataset = ImageFolder(os.path.join(root_dir, "test"), transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size = 8, num_workers=4, shuffle=True)
test_loader   = DataLoader(test_dataset,   batch_size = 8, num_workers=4, shuffle=False)
loaders = {
    "train": train_loader,
    "test": test_loader
}

In [8]:
num_classes = len(train_dataset.classes)
print(num_classes)

8


In [0]:
class CapsNet(nn.Module):
  # Constructor
  def __init__(self):
    # Call parent constructor
    super().__init__()
    # Create convolutional layers
    self.conv_layers = nn.Sequential(
      # Layer 1
      nn.Conv2d(3, 16, kernel_size=2, padding=0, stride=2),
      nn.ReLU()
    )
  
  # Forward
  def forward(self, x):
    x = self.conv_layers(x)
    x = x.unsqueeze(2)
    return x

In [14]:
batch,labels = next(iter(train_loader))
batch = batch.to(dev)
labels = labels.to(dev)
model = CapsNet()
model = model.to(dev)
out = model(batch)
print(out.size())

torch.Size([8, 16, 1, 128, 128])
